In [10]:
import tensorflow as tf
import os
from pathlib import Path
import urllib.request as req
import numpy as np
from zipfile import ZipFile
import matplotlib.pyplot as plt
import tensorflow as tf

In [11]:
data_URL = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

In [12]:
def create_dir(Data_dir):
    os.makedirs(Data_dir,exist_ok=True)
    print(f'{Data_dir} created')
Root_dir='hymenoptera_data'

create_dir(Root_dir)

hymenoptera_data created


In [13]:
data_path=os.path.join(Root_dir,"data.zip")

if not os.path.isfile(data_path):
    print("downloading data...")
    filename,headers=req.urlretrieve(data_URL,data_path)
    print(f"filename: {filename} created with info \n{headers}")
else:
    print(f"file is already present")

file is already present


In [14]:


unzip_data_dir=os.path.join(Root_dir,'unzip_data')

In [15]:
from zipfile import ZipFile

unzip_data_dir=os.path.join(Root_dir,'unzip_data')

if not os.path.exists(unzip_data_dir):
    create_dir(unzip_data_dir)
    with ZipFile(data_path) as zip_:
        print(f'data extraction started')
        zip_.extractall(unzip_data_dir)
else:
    print(f'Data already extracted')

hymenoptera_data\unzip_data created
data extraction started


In [19]:
main_data_dir=Path("./hymenoptera_data/unzip_data/hymenoptera_data/train")

In [20]:
main_data_dir

WindowsPath('hymenoptera_data/unzip_data/hymenoptera_data/train')

In [66]:
BATCH_SIZE=32
Image_size=(224,224)


In [67]:
data_gen_kwargs=dict(rescale=1./255,validation_split=0.20)

In [68]:
dataflow_kwargs=dict(target_size=Image_size,batch_size=BATCH_SIZE)

In [69]:
valid_data_gen=tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_kwargs)

In [70]:
valid_dataflow=valid_data_gen.flow_from_directory(main_data_dir,subset='validation'
                                                  ,shuffle=False,**dataflow_kwargs)

Found 48 images belonging to 2 classes.


In [71]:
valid_data_gen=tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_kwargs)
do_data_augmentation=False
if do_data_augmentation:
    train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,**data_gen_kwargs)
else:
    train_datagen=valid_data_gen

train_dataflow=train_datagen.flow_from_directory(main_data_dir,subset='training',
                                                shuffle=False,**dataflow_kwargs)

Found 196 images belonging to 2 classes.


In [72]:
RGB_IMAGE_SIZE=(224,224,3)

In [73]:
vgg=tf.keras.applications.vgg16.VGG16(
input_shape=RGB_IMAGE_SIZE,
weights='imagenet',
include_top=False)

In [74]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [75]:
for layer in vgg.layers:
    layer.trainable=False
    print(f'{layer.name}:{layer.trainable}')

input_3:False
block1_conv1:False
block1_conv2:False
block1_pool:False
block2_conv1:False
block2_conv2:False
block2_pool:False
block3_conv1:False
block3_conv2:False
block3_conv3:False
block3_pool:False
block4_conv1:False
block4_conv2:False
block4_conv3:False
block4_pool:False
block5_conv1:False
block5_conv2:False
block5_conv3:False
block5_pool:False


In [76]:
vgg.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [77]:
CLASSES=2

x=tf.keras.layers.Flatten()(vgg.output)
prediction=tf.keras.layers.Dense(CLASSES,activation='softmax')(x)

In [78]:
model=tf.keras.models.Model(inputs=vgg.input,outputs=prediction)

In [79]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [80]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01,momentum=0.9),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [81]:
epochs=10

history=model.fit(train_dataflow,epochs=epochs,validation_data=valid_dataflow,
                 )

Epoch 1/10
7/7 [==============================] - 25s 3s/step - loss: 60.9547 - accuracy: 0.3622 - val_loss: 32.1414 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 24s 4s/step - loss: 58.9598 - accuracy: 0.3418 - val_loss: 1.1557 - val_accuracy: 0.8958
Epoch 3/10
7/7 [==============================] - 25s 4s/step - loss: 17.8075 - accuracy: 0.7500 - val_loss: 36.7992 - val_accuracy: 0.5000
Epoch 4/10
7/7 [==============================] - 24s 4s/step - loss: 77.4772 - accuracy: 0.5255 - val_loss: 124.2625 - val_accuracy: 0.5000
Epoch 5/10
7/7 [==============================] - 24s 3s/step - loss: 64.7872 - accuracy: 0.6990 - val_loss: 108.1103 - val_accuracy: 0.5000
Epoch 6/10
7/7 [==============================] - 25s 4s/step - loss: 139.0028 - accuracy: 0.6378 - val_loss: 109.3468 - val_accuracy: 0.5000
Epoch 7/10
7/7 [==============================] - 24s 3s/step - loss: 274.7729 - accuracy: 0.4949 - val_loss: 89.5323 - val_accuracy: 0.5000
Epoch 8/10
7/7 [